# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [ ]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com/"

# assign the /status endpoint to another variable called status_url (1 line)
status_url ="https://country-leaders.onrender.com/status/"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
response = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
response_code = response.status_code
if response_code == 200:
    print(f"{response_code}: OK, request succesful")
else:
    print("ERROR. Something went wrong")


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [ ]:
# Set the countries_url variable (1 line)
countries_url = root_url+ "countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
countries_resp = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
print(countries_resp.json)

# display the request's status code and the countries variable (1 line)
print(countries_resp.status_code)

### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [ ]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + "cookie/"

# Query the enpoint, set the cookies variable and display it (2 lines)
cookie_resp = requests.get(cookie_url)
user_cookie = cookie_resp.cookies['user_cookie']

Try to query the countries endpoint using the cookie, save the output and print it.

In [ ]:
def call_user_cookie():
    cookie_url = root_url + "cookie/"
    cookie_resp = requests.get(cookie_url)
    return {'user_cookie': cookie_resp.cookies['user_cookie']}

In [ ]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

countries_resp = requests.get(countries_url, cookies=call_user_cookie())

# display the countries variable (1 line)
print(countries_resp.json)



In [ ]:
countries = countries_resp.json()

Chances are the cookie has expired... Thankfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [ ]:
# Set the leaders_url variable (1 line)
leaders_url = root_url+"/leaders"
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders_resp = requests.get(leaders_url, cookies=call_user_cookie())

# display the leaders variable (1 line)
print(leaders_resp.content)


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders_resp = requests.get(leaders_url, cookies=call_user_cookie(), params={'country':'be'})
# display the leaders variable (1 line)
print(leaders_resp.content)

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leaders_per_country =dict()
for country in countries:
    country_resp = requests.get(leaders_url, cookies=call_user_cookie(), params={'country':country})
    leaders_per_country[country]=country_resp.content


In [ ]:
for k, v in leaders_per_country.items():
    print(f"For country {k}: {v}")

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [ ]:
import requests
import json
from bs4 import BeautifulSoup 

In [ ]:
# < 15 lines
def get_leaders():
    # Establishing the urls
    root_url = "https://country-leaders.onrender.com/"
    cookie_url = root_url+"cookie/"
    countries_url = root_url + "countries"
    leaders_url = root_url+"/leaders"
    leaders_per_country =dict()

    # Getting the cookie
    cookie_resp = requests.get(cookie_url)
    user_cookie = cookie_resp.cookies['user_cookie']

    countries_resp = requests.get(countries_url, cookies={'user_cookie':user_cookie})
    countries = countries_resp.json()
    
    for country in countries:
        
        country_resp = requests.get(leaders_url, cookies={'user_cookie':user_cookie}, params={'country':country})
        leaders_per_country[country]=country_resp.text
    
    return leaders_per_country
    


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
leaders_p_country = get_leaders()

for k, v in leaders_p_country.items():
    print(f"For country {k}: {v}")

In [ ]:
## To do
### Function to extract the wikipedia_url

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [ ]:
# 3 lines
dict_us = leaders_p_country['us']
dict_ma = leaders_p_country['ma']
dict_fr = leaders_p_country['fr']

In [ ]:
print(dict_us)
print(dict_ma)
print(dict_fr)

In [ ]:
# Convert string to dictionary
ma_json = json.loads(dict_ma)
fr_json = json.loads(dict_fr)

wikipedia_url_ma =ma_json[1]["wikipedia_url"]
wikipedia_url_fr =fr_json[1]["wikipedia_url"]

In [ ]:
leaders_wiki_ma = requests.get(wikipedia_url_ma)
leaders_wiki_fr = requests.get(wikipedia_url_fr)

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
soup_fr = BeautifulSoup(leaders_wiki_fr.content, "html")
soup_fr

In [ ]:
soup_ma = BeautifulSoup(leaders_wiki_ma.content, "html")
soup_ma

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
# 2 lines
for p in soup.find_all("p"):
    print(p.text)

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [ ]:
dict_us = leaders_p_country['us']
us_json = json.loads(dict_us)
wikipedia_url_us = us_json[0]["wikipedia_url"]
print(wikipedia_url_us)
leaders_wiki_us = requests.get(wikipedia_url_us)
soup_us = BeautifulSoup(leaders_wiki_us.content, "html")
soup_us
parrag_us=[]
for p in soup_us.find_all("p"):
    parrag_us.append(p)

for p in parrag_us:
    # Find the first instance of bold. It should be the name
    if p.find("b"): 
        print(p.text)
        # so it only takes the first bold instance
        break 

##### Testing with every country

In [ ]:
dict_fr = leaders_p_country['fr']
fr_json = json.loads(dict_fr)
wikipedia_url_fr = fr_json[0]["wikipedia_url"]
print(wikipedia_url_fr)
leaders_wiki_fr = requests.get(wikipedia_url_fr)
soup_fr = BeautifulSoup(leaders_wiki_fr.content, "html")
soup_fr
parrag_fr=[]
for p in soup_fr.find_all("p"):
    parrag_fr.append(p)

for p in parrag_fr:
    if p.find("b"):
        print(p.text)
        break

In [ ]:
dict_be = leaders_p_country['be']
be_json = json.loads(dict_be)
wikipedia_url_be = be_json[0]["wikipedia_url"]
print(wikipedia_url_be)
leaders_wiki_be = requests.get(wikipedia_url_be)
soup_be = BeautifulSoup(leaders_wiki_be.content, "html")
soup_be
parrag_be=[]
for p in soup_be.find_all("p"):
    parrag_be.append(p)

for p in parrag_be:
    if p.find("b"):
        print(p.text)
        break

In [ ]:
dict_ma = leaders_p_country['ma']
ma_json = json.loads(dict_ma)
wikipedia_url_ma =ma_json[0]["wikipedia_url"]
print(wikipedia_url_ma)
leaders_wiki_ma = requests.get(wikipedia_url_ma)
soup_ma = BeautifulSoup(leaders_wiki_ma.content, "html")
soup_ma
parrag_ma=[]
for p in soup_ma.find_all("p"):
    parrag_ma.append(p)

for p in parrag_ma:
    if p.find("b"):
        print(p.text)
        break

In [ ]:
dict_ru = leaders_p_country['ru']
ru_json = json.loads(dict_ru)
wikipedia_url_ru =ru_json[0]["wikipedia_url"]
print(wikipedia_url_ru)
leaders_wiki_ru = requests.get(wikipedia_url_ru)
soup_ru = BeautifulSoup(leaders_wiki_ru.content, "html")

parrag_ru=[]
for p in soup_ru.find_all("p"):
    parrag_ru.append(p)
    
for p in parrag_ru:
    if p.find("b"):
        print(p.text)
        break

### Function

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [ ]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

    leaders_wiki = requests.get(wikipedia_url)
    soup_leaders = BeautifulSoup(leaders_wiki .content, "html")
   
    parragraphs=[]
    for p in soup_leaders.find_all("p"):
        parragraphs.append(p)

    for p in parragraphs:
        # Find the first instance of bold. It should be the name
        if p.find("b"): 
            return p.text
            # so it only takes the first bold instance

In [ ]:
get_first_paragraph(wikipedia_url_us)

In [ ]:
get_first_paragraph(wikipedia_url_fr)

In [ ]:
get_first_paragraph(wikipedia_url_be)

In [ ]:
get_first_paragraph(wikipedia_url_ma)

In [ ]:
get_first_paragraph(wikipedia_url_ru)

### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re
# ### things to get rid of:
##/xa0
## ['number']

pattern = '\S*ⓘ\S*|/xa0|\[.{0,3}?\]'

wiki_urls= [wikipedia_url_us, wikipedia_url_be, wikipedia_url_fr, wikipedia_url_ma, wikipedia_url_ru]

for url in wiki_urls:
    s = get_first_paragraph(url)
    print("-------LEADER------")
    print(s)
    
    clean_p = re.sub(pattern,'',s)

    print(clean_p)


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

    leaders_wiki = requests.get(wikipedia_url)
    soup_leaders = BeautifulSoup(leaders_wiki .content, "html")
   
    parragraphs=[]
    for p in soup_leaders.find_all("p"):
        parragraphs.append(p)

    for p in parragraphs:
        # Find the first instance of bold. It should be the name
        if p.find("b"): 
            raw_first_p = p.text
            # so it only takes the first bold instance
        
    pattern = '\S*ⓘ\S*|/xa0|\[.{0,3}?\]'

    clean_first_p = re.sub(pattern,'',raw_first_p)

    return clean_first_p


In [ ]:
wiki_urls= [wikipedia_url_us, wikipedia_url_be, wikipedia_url_fr, wikipedia_url_ma, wikipedia_url_ru]

for url in wiki_urls:
    s = get_first_paragraph(url)
    print(s)

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [ ]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook

    leaders_wiki = requests.get(wikipedia_url)
    soup_leaders = BeautifulSoup(leaders_wiki .content, "html")
   
    parragraphs=[]
    for p in soup_leaders.find_all("p"):
        parragraphs.append(p)

    for p in parragraphs:
        # Find the first instance of bold. It should be the name
        if p.find("b"): 
            raw_first_p = p.text
            # so it only takes the first bold instance
            break
        
    pattern = '\S*ⓘ\S*|/xa0|\[.{0,3}?\]'

    clean_first_p = re.sub(pattern,'',raw_first_p)

    return clean_first_p


In [ ]:
def get_leaders():
    # Establishing the urls
    root_url = "https://country-leaders.onrender.com/"
    cookie_url = root_url+"cookie/"
    countries_url = root_url + "countries"
    leaders_url = root_url+"/leaders"
    leaders_per_country = dict()

    # Getting the cookie
    cookie_resp = requests.get(cookie_url)
    user_cookie = cookie_resp.cookies['user_cookie']

    countries_resp = requests.get(countries_url, cookies={'user_cookie':user_cookie})
    countries = countries_resp.json()
    
    for country in countries:
        country_resp = requests.get(leaders_url, cookies={'user_cookie':user_cookie}, params={'country':country})
        print(country_resp )
        leaders_per_country[country]=country_resp.text

    all_leaders = []

    # Obtaining the Wikipedia url
    for country, leaders in leaders_per_country.items():
        leaders_dict = json.loads(leaders)
        for leaders in leaders_dict:
            leaders['country'] = country
            all_leaders.append(leaders)
    
 
    for leader in all_leaders:
        wiki_parragraph = get_first_paragraph(leader['wikipedia_url'])
        leader['wiki_parragraph'] = wiki_parragraph
     
               
    return all_leaders

In [ ]:
l = get_leaders()

In [ ]:
print(l)

### Delete this cell

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines
def get_leaders():
    # Establishing the urls
    root_url = "https://country-leaders.onrender.com/"
    cookie_url = root_url+"cookie/"
    countries_url = root_url + "countries"
    leaders_url = root_url+"/leaders"
    leaders_per_country = dict()

    # Getting the cookie
    cookie_resp = requests.get(cookie_url)
    user_cookie = cookie_resp.cookies['user_cookie']

    countries_resp = requests.get(countries_url, cookies={'user_cookie':user_cookie})

    if cookie_resp.status_code != 200:
        cookie_resp = requests.get(cookie_url)
        user_cookie = cookie_resp.cookies['user_cookie']
        countries_resp = requests.get(countries_url, cookies={'user_cookie':user_cookie})
        
    countries = countries_resp.json()
    
    for country in countries:
        country_resp = requests.get(leaders_url, cookies={'user_cookie':user_cookie}, params={'country':country})
        print(country_resp )
        leaders_per_country[country]=country_resp.text

    all_leaders = []

    # Obtaining the Wikipedia url
    for country, leaders in leaders_per_country.items():
        leaders_dict = json.loads(leaders)
        for leaders in leaders_dict:
            leaders['country'] = country
            all_leaders.append(leaders)

    for leader in all_leaders:
        wiki_parragraph = get_first_paragraph(leader['wikipedia_url'])
        leader['wiki_parragraph'] = wiki_parragraph
     
               
    return all_leaders


Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
get_leaders()

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines
def get_first_paragraph(wikipedia_url, session):
    print(wikipedia_url) # keep this for the rest of the notebook
    
    leaders_wiki = session.get(wikipedia_url)
    soup_leaders = BeautifulSoup(leaders_wiki.content, "html")
   
    parragraphs=[]
    for p in soup_leaders.find_all("p"):
        parragraphs.append(p)

    for p in parragraphs:
        # Find the first instance of bold. It should be the name
        if p.find("b"): 
            raw_first_p = p.text
            # so it only takes the first bold instance
            break
    
    #Clean the text with regex
        
    pattern = '\\n|\S*ⓘ\S*|xa0|\[.{0,3}?\]|/'

    clean_first_p = re.sub(pattern,'',raw_first_p)

    return clean_first_p


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines
def get_leaders():
    # Establishing the urls
    root_url = "https://country-leaders.onrender.com/"
    cookie_url = root_url+"cookie/"
    countries_url = root_url + "countries"
    leaders_url = root_url+"/leaders"
    leaders_per_country = dict()

    # Getting the cookie
    cookie_resp = requests.get(cookie_url)
    user_cookie = cookie_resp.cookies['user_cookie']
    
    countries_resp = requests.get(countries_url, cookies={'user_cookie':user_cookie})

    if cookie_resp.status_code != 200:
        cookie_resp = requests.get(cookie_url)
        user_cookie = cookie_resp.cookies['user_cookie']
        countries_resp = requests.get(countries_url, cookies={'user_cookie':user_cookie})
        
    countries = countries_resp.json()
    
    for country in countries:
        country_resp = requests.get(leaders_url, cookies={'user_cookie':user_cookie}, params={'country':country})
        print(country_resp )
        leaders_per_country[country]=country_resp.text

    all_leaders =[]

    # Obtaining the Wikipedia url
    for country, leaders in leaders_per_country.items():
        leaders_dict = json.loads(leaders)
        for leaders in leaders_dict:
            leaders['country'] = country
            all_leaders.append(leaders)

    #s= requests.Session()

    #for leader in all_leaders:
        #wiki_parragraph = get_first_paragraph(leader['wikipedia_url'], s)
        #leader['wiki_parragraph'] = wiki_parragraph
    
    with requests.Session() as s:
        for leader in all_leaders:
            wiki_parragraph = get_first_paragraph(leader['wikipedia_url'], s)
            leader['wiki_parragraph'] = wiki_parragraph
        
               
    return all_leaders

Test your new functions.



In [ ]:
leaders_one_session = get_leaders()
print(leaders_one_session)

In [ ]:
print(leaders_one_session)

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
json_leaders = json.dumps(leaders_one_session, ensure_ascii=False, indent =4)

with open("leaders.json", "w", encoding='utf-8') as outfile:
    outfile.write(json_leaders)

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
with open ("leaders.json", "r", encoding='utf-8') as file:
    content = file.read()

print(content)

Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
def save(leaders_per_country):
    json_leaders = json.dumps(leaders_per_country, ensure_ascii=False, indent =4)

    with open("leaders.json", "w", encoding='utf-8') as outfile:
        outfile.write(json_leaders)
    
    with open ("leaders.json", "r", encoding='utf-8') as file:
        content = file.read()

    print("The following content has been saved to 'leaders.json' :")
    print(content)

In [ ]:
# Call the function (1 line)
save(leaders_one_session)

## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!

#### Missing personal details

In [ ]:
leaders_data = get_leaders()

In [ ]:
## Identifying which keys have missing information

for leader in leaders_data:
    for key, value in leader.items():
        if not value:
            print(f"Leader ID: {leader['id']}: {key} : {value}")

In [ ]:
missing_data_leaders = []

In [ ]:
from datetime import datetime

In [ ]:

def get_missing_info(leader_data, session): 
    print(leader_data['wikipedia_url'])
    for key, value in leader_data.items():
        if not value:
            value_to_find = key
            
    
    leader_wiki = session.get(leader_data['wikipedia_url'])

    soup_leader = BeautifulSoup(leader_wiki.content, "html")

    text = ""
    
    
    # For English text:
    if value_to_find == 'birth_date':
        #find the birth data in the soup
        text ='Born'
        key_to_fill = value_to_find
        
    elif value_to_find == 'death_date':
        # find the death data in the soup    
        text = 'Died'
        key_to_fill = value_to_find
    
    else:
        return

    for i in soup_leader.find_all('th', text=text):
        print("Found!")
        print("-------------")
        missing_raw_info = i.find_next().text
        print(missing_raw_info)
        date_pattern = r'([A-Z][a-z]+) \s*(\d{1,2}),\s*(\d{4})'
        match = re.search(date_pattern, missing_raw_info)
        print(match.group())
        
        date_obj = datetime.strptime(match.group(), "%B %d, %Y").date()
        
        print(date_obj, type(date_obj))
        leader_data[key_to_fill] = str(date_obj)
    return leader_data
    

In [47]:
s = requests.Session()

for leader in leaders_data:
    for key, value in leader.items():
        if not value:
        
            get_missing_info(leader,s)

https://en.wikipedia.org/wiki/Joe_Biden


KeyboardInterrupt: 